# Pre Selection - Deployment

Select the features according to the following criteria:
- Features with variability close to 0
- Features correlated between each other
- Handle NaN and missing values 

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Wrapping Custom Transformer

In [ ]:
%%writefile CustomTransformer.py
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class Correlation(BaseEstimator, TransformerMixin):
    """Feature selector that removes correlated features.
    
    This feature selection algorithm looks only at features(X) and 
    and removes those with high correlation.
    
    Attributes:
        categorical_indexes: a np.ndarray of categoricals indexes.
        cutoff: float of cutoff.
        drop_indexes: list of indexes to be droped.
    """

    def __init__(self, categorical_indexes: np.ndarray, cutoff: float):
        """Inits Correlation class.
        
        Args:
            categorical_indexes: categorical indexes of X.
            cutoff: cutoff value.
        """
        self.categorical_indexes = categorical_indexes
        self.cutoff = cutoff
    
    def transform(self, X) -> np.ndarray:
        """Reduce X to the selected features.
        
        Args:
            X: the input samples.
            
        Returns:
            np.ndarray: the input samples without only the selected features.
        """
        return np.delete(X, np.unique(self.drop_indexes), axis=1)
    
    def get_support(self) -> np.ndarray:
        """Returns a list of indexes to be removed.
        
        Returns:
            np.ndarray: indexes removed by the model.
        """
        return np.unique(self.drop_indexes)
    
    def fit(self, X: np.ndarray, y=None) -> np.ndarray:
        """Fit the model.
        
        Learn correlated features from X.
        
        Args:
            X: the imput sample.
        
        Returns:
            self
        """
        # get only numerical values from X
        X_numerical = np.delete(X, self.categorical_indexes, axis=1)
        
        # check the shape of input
        if np.ma.size(X_numerical, axis=0) <= 1 \
        or np.ma.size(X_numerical, axis=1) <= 1:
            return X
        
        # correlation matrix
        corr_matrix = np.abs(np.corrcoef(X_numerical.astype(float), rowvar=False))
        
        # mean correlation for each column
        mean_corr = np.mean(corr_matrix, axis=1)

        # pairwise correlations above cutoff
        above_cutoff = np.argwhere(np.triu(corr_matrix, k=1) > self.cutoff)

        # for each pairwise correlation above cutoff
        # remove the feature with the highest mean correlation 
        self.drop_indexes = [
            above_cutoff[i, np.argmax(pair)] 
            for i, pair in enumerate(mean_corr[above_cutoff])]

        return self

## Wrapping Model

Allows your component to expose a service over REST.

To wrap your model [follow the instructions](https://docs.seldon.io/projects/seldon-core/en/v0.3.0/python/python_component.html) for your chosen language or toolkit.

In [ ]:
%%writefile Model.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Model(object):    
    def __init__(self, dataset: str = None, target: str = None):
        # Load Artifacts: Estimator, etc
        model = load_model()
        self.pipeline = model["pipeline"]
        self.features_names_training = model["columns"]
        self.features_after_pipeline = model["features_after_pipeline"]

    def class_names(self):
        return self.features_after_pipeline.tolist()

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        """Takes an array (numpy) X and feature_names and completes missing values.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.
            meta (dict, optional): Dict of metadata.
        """
        if feature_names:
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.features_names_training].to_numpy()
            
        X = self.pipeline.transform(X)

        return X

## API Contract

There are two sections:

- `features` : The feature array you intend to send in a request
- `targets` : The response you expect back

Each section has a list of definitions. Each definition consists of:

- `name` : String : The name of the feature
- `ftype` : one of CONTINUOUS, CATEGORICAL : the type of the feature
- `dtype` : One of FLOAT, INT : Required for ftype CONTINUOUS : What type of feature to create
- `values` : list of Strings : Required for ftype CATEGORICAL : The possible categorical values
- `range` : list of two numbers : Optional for ftype CONTINUOUS : The range of values (inclusive) that a continuous value can take
- `repeat` : integer : Optional value for how many times to repeat this value
- `shape` : array of integers : Optional value for the shape of array to coerce the values

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "SepalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [4.3, 7.9]
        },
        {
            "name": "SepalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [2.0, 4.4]
        },
        {
            "name": "PetalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 6.9]
        },
        {
            "name": "PetalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.1, 2.5]
        }
    ],
    "targets": [
        {
            "name": "SepalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [4.3, 7.9]
        },
        {
            "name": "SepalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [2.0, 4.4]
        },
        {
            "name": "PetalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.1, 2.5]
        }
    ]
}

## Test Deployment

Starts a service wrapping a Model, sends a request to the service, and shows the response.

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")